In [ ]:
from pycaret.time_series import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycaret.utils.time_series import clean_time_index
import holidays
from scipy.stats import boxcox
from scipy.special import inv_boxcox

# อ่านข้อมูลจากไฟล์ CSV
df = pd.read_csv("cleaned_data.csv")

# ลบคอลัมน์ที่ไม่จำเป็น (ถ้ามี)
df = df.drop(columns=["Unnamed: 0"], errors='ignore')

# แปลงคอลัมน์ 'timestamp' เป็น string และตัดมิลลิวินาทีออก
df['timestamp'] = df['timestamp'].astype(str).str.split('.').str[0]

# แปลงคอลัมน์ 'timestamp' เป็น datetime และตั้งเป็น index
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# ตรวจสอบว่ามีค่า NaT (ไม่สามารถแปลงได้) หรือไม่
if df['timestamp'].isna().any():
    print("พบ timestamp ที่ไม่สามารถแปลงได้:")
    print(df[df['timestamp'].isna()])
    # ลบแถวที่มีค่า NaT
    df = df.dropna(subset=['timestamp'])

df = df.set_index('timestamp')

# ตรวจสอบว่า index เป็น unique หรือไม่
print("Index เป็น unique หรือไม่:", df.index.is_unique)

# ลบ index ที่ซ้ำกัน
df = df[~df.index.duplicated(keep="last")]

# ตั้งค่าความถี่เป็นรายชั่วโมง (H) และเติมค่าหายไป
df = df.asfreq("H")

# ลบแถวที่มี timestamp ในช่วงที่ต้องการลบ
timestamps_to_drop = pd.to_datetime([
    '2025-02-11 00:00:00', '2025-02-11 01:00:00', '2025-02-11 02:00:00',
    '2025-02-11 03:00:00', '2025-02-11 04:00:00', '2025-02-11 05:00:00',
    '2025-02-11 06:00:00', '2025-02-11 07:00:00', '2025-02-11 08:00:00',
    '2025-02-11 09:00:00', '2025-02-11 10:00:00', '2025-02-11 11:00:00',
    '2025-02-11 11:12:42'
])
df = df[~df.index.isin(timestamps_to_drop)]
df = df.interpolate(method='polynomial', order=2)

# กรองค่า pm_2_5 ให้อยู่ในช่วง 0 ถึง 80
df = df[(df['pm_2_5'] >= 0) & (df['pm_2_5'] <= 80)]

# ใช้เฉพาะคอลัมน์ที่ต้องการ (pm_2_5)
df = df[['pm_2_5']]

# เพิ่มฟีเจอร์ วัน, เดือน, ปี
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year

# เพิ่มฟีเจอร์ "ฤดูกาล" ของประเทศไทย
def get_thai_season(month):
    if month in [3, 4, 5]:
        return 'summer'
    elif month in [6, 7, 8, 9, 10]:
        return 'rainy'
    else:
        return 'winter'

df['season'] = df['month'].apply(get_thai_season)

# One-hot encoding สำหรับฤดูกาล
df = pd.get_dummies(df, columns=['season'], prefix='season')

# เพิ่มฟีเจอร์ "วันหยุด" ของประเทศไทย
thai_holidays = holidays.TH(years=df.index.year.unique())
df['is_holiday'] = df.index.to_series().apply(lambda x: 1 if x in thai_holidays else 0)

# แปลงประเภทข้อมูลของคอลัมน์ให้เป็นตัวเลข
df['is_holiday'] = df['is_holiday'].astype(int)
df['season_summer'] = df['season_summer'].astype(int)
df['season_rainy'] = df['season_rainy'].astype(int)
df['season_winter'] = df['season_winter'].astype(int)

# แบ่งข้อมูลเป็น train set และ test set
train_size = len(df) - 168
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

# เติมค่าหายไปใน train set
train_df = train_df.fillna(train_df.mean())

# แปลงข้อมูลด้วย Box-Cox Transformation
train_df['pm_2_5'], lambda_ = boxcox(train_df['pm_2_5'] + 1)  # +1 เพื่อหลีกเลี่ยงค่าติดลบ

##เช็๕ข้อมู,ใน trian
print(df)
train_df = train_df.asfreq("H")
train_df = train_df.interpolate(method='polynomial', order=2)
test_df = test_df.asfreq("H")
test_df = test_df.interpolate(method='polynomial', order=2)
print(train_df.index.freq)

# ตั้งค่า PyCaret
exp = TSForecastingExperiment()
exp.setup(
    data=train_df, 
    target='pm_2_5', 
    session_id=123, 
    fh=168, 
    use_gpu=True, 
    seasonal_period=12 
)

# สร้างโมเดล ARIMA
model = exp.create_model('arima', order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
model = exp.tune_model(model)
model = exp.finalize_model(model)

# ทำนายค่า pm_2_5
forecast = exp.predict_model(model, fh=168, X=test_df.drop(columns="pm_2_5"))

# แปลงค่าทำนายกลับด้วย inverse_boxcox
forecast['y_pred'] = inv_boxcox(forecast['y_pred'], lambda_) - 1

# ปรับค่าทำนายให้ไม่เป็นลบ
forecast['y_pred'] = np.maximum(forecast['y_pred'], 0)

# แสดงผลลัพธ์
print("Forecast:")
print(forecast)

# แก้ไขการพล็อตกราฟเปรียบเทียบค่าจริงกับค่าทำนาย
plt.figure(figsize=(10, 6))
plt.plot(df.index[-50:].to_timestamp(), df['pm_2_5'][-50:], label='Actual', marker='o')
plt.plot(forecast.index.to_timestamp(), forecast['y_pred'], label='Forecast', marker='s', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('pm_2_5')
plt.title('Actual vs Forecasted pm_2_5')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Index เป็น unique หรือไม่: True
                        pm_2_5  day  month  year  season_rainy  season_summer  \
timestamp                                                                       
2022-03-18 05:00:00  21.200000   18      3  2022             0              1   
2022-03-18 06:00:00  12.842975   18      3  2022             0              1   
2022-03-18 07:00:00  18.376147   18      3  2022             0              1   
2022-03-18 08:00:00  23.533333   18      3  2022             0              1   
2022-03-18 09:00:00  19.066667   18      3  2022             0              1   
...                        ...  ...    ...   ...           ...            ...   
2025-02-10 19:00:00  18.834711   10      2  2025             0              0   
2025-02-10 20:00:00  24.819672   10      2  2025             0              0   
2025-02-10 21:00:00  21.596639   10      2  2025             0              0   
2025-02-10 22:00:00  20.200000   10      2  2025             0              0

,Description,Value
0,session_id,123
1,Target,pm_2_5
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(25291, 8)"
5,Transformed data shape,"(25291, 8)"
6,Transformed train set shape,"(25123, 8)"
7,Transformed test set shape,"(168, 8)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,01:04:58
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,ARIMA


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(test_df)

               pm_2_5  day  month  year  season_rainy  season_summer  \
2025-01-28  10.800574   28      1  2025             0              0   
2025-01-29  10.605639   29      1  2025             0              0   
2025-01-30  14.708435   30      1  2025             0              0   
2025-01-31  16.408323   31      1  2025             0              0   
2025-02-01  27.336896    1      2  2025             0              0   
2025-02-02  30.186564    2      2  2025             0              0   
2025-02-03  30.616034    3      2  2025             0              0   
2025-02-04  18.877155    4      2  2025             0              0   
2025-02-05   6.793629    5      2  2025             0              0   
2025-02-06  10.463530    6      2  2025             0              0   
2025-02-07  19.059025    7      2  2025             0              0   
2025-02-08  19.064854    8      2  2025             0              0   
2025-02-09  13.824501    9      2  2025             0           

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ดึงค่าจริงจาก test_df
actual = test_df['pm_2_5']

# ดึงค่าทำนายจาก forecast
predicted = forecast['y_pred']

# คำนวณ MAE
mae = mean_absolute_error(actual, predicted)

# คำนวณ MSE
mse = mean_squared_error(actual, predicted)

# คำนวณ RMSE
rmse = np.sqrt(mse)

# คำนวณ MAPE
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

mape = calculate_mape(actual, predicted)

# คำนวณ R²
r2 = r2_score(actual, predicted)

# คำนวณความแม่นยำ (Accuracy)
mean_actual = np.mean(actual)
accuracy = (1 - (mae / mean_actual)) * 100

# แสดงผลลัพธ์
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(r2)
print(f"R² (ความแม่นยำ): {r2 * 100:.2f}%")
print(f"ความแม่นยำ (Accuracy): {accuracy:.2f}%")

MAE: 5.4474
MSE: 47.0706
RMSE: 6.8608
MAPE: 37.70%
0.09595384725230871
R² (ความแม่นยำ): 9.60%
ความแม่นยำ (Accuracy): 70.12%
